In [7]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import time
import sys
import warnings
warnings.filterwarnings('ignore')

# generate url for product
def get_url(product):
    product = product.replace(' ','%20')
    template = 'https://shopee.com.my/search?keyword={}/'
    url = template.format(product)
    return url  

def get_all_products(card):
    
    # if use selenium
    # product_image = single_card.find_element_by_tag_name('img')
    # productImg = product_image.get_attribute('src')
    
    # if use BeautifulSoup
    pImg = card.find('img','_3-N5L6 _2GchKS')
    product_image = pImg['src']
    product_name = card.find('div','_10Wbs- _5SSWfi UjjMrh').text.strip()
    product_name = product_name.encode('ascii','ignore')
    product_name = str(product_name,'utf-8')
    product_price = card.find('div','zp9xm9 xSxKlK _1heB4J').text.strip()
    anchor_tag = card.a.get('href')
    product_buy_link = 'https://shopee.com.my/'+anchor_tag
    
    product_info = (product_image,product_name,product_price,product_buy_link)
    
    return product_info

def main(product):
    records = []
    url = get_url(product)
    
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get(url)
    driver.maximize_window()
    # wait for 5 seconds
    time.sleep(5)
    btn = driver.find_element_by_xpath('//*[@id="modal"]/div[1]/div[1]/div/div[3]/div[1]/button')
    btn.click()
    
    #Define an initial value
    temp_height=0
    while True:
        #Looping down the scroll bar
        driver.execute_script("window.scrollBy(0,1000)")
        #sleep and let the scroll bar react
        time.sleep(5)
        #Get the distance of the current scroll bar from the top
        check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
        #If the two are equal to the end
        if check_height==temp_height:
            break
        temp_height=check_height
        
    time.sleep(10) 
    
    # create a BeautifulSoup object
    soup = BeautifulSoup(driver.page_source,'html.parser')
    product_cards = soup.find_all('div','col-xs-2-4')
    
    # fetch single product from shopee
    singleCard = product_cards[0]
    productDetails = get_all_products(singleCard)
    
    return productDetails

pname = sys.argv[1]
pDetails = main(pname) 
print(pDetails)

('https://cf.shopee.com.my/file/aa0a8cb3ad16fa1ca683fa1bc382be03_tn', 'F&N 100Plus (24 x 325ml) X 2 Carton [KL & Selangor Delivery Only]', 'RM63.00', 'https://shopee.com.my//F-N-100Plus-(24-x-325ml)-X-2-Carton-KL-Selangor-Delivery-Only--i.3558759.3810317008?sp_atk=e7ee4f07-8461-4cdf-9bf7-8cc806f4638a')
